In [49]:
import skimage.morphology as skmorph
import os
import datetime as dt
import numpy as np
import pandas as pd
import scipy.ndimage as ndimage

from pygridder import Gridder
import pygrib as pg
import pathlib

In [4]:
dx = 5 # delta x
selem = skmorph.disk(40 / dx) # morphology disk

In [5]:
ndfd_path = pathlib.Path('..','..','impacts-data','pas-input-data','ndfd.npz').resolve()

In [6]:
with np.load(ndfd_path) as NPZ:
    lons = NPZ['lons']
    lats = NPZ['lats']

In [ ]:
G = Gridder(tx=lons, ty=lats, dx=dx/100)

### Read in onetor data and prep

In [7]:
data_path = pathlib.Path('..','raw-tor-data','1950-2018_actual_tornadoes.csv')

In [8]:
dateparser = lambda x: dt.datetime.strptime(x, '%Y-%m-%d %H:%M:%S') + dt.timedelta(hours=6)

In [9]:
df = pd.read_csv(data_path, parse_dates=[['date','time']], date_parser=dateparser, index_col=0, keep_date_col=True)

In [10]:
df = df[df.sg == 1]
df = df.reset_index()

In [11]:
di = {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0}
df['weight'] = df['mag'].map(di).fillna(df['mag'])

In [15]:
outlook_time = '1200'

In [ ]:
# Create date array
my_hour = dt.datetime.strptime(outlook_time, '%H%M').hour
my_minute = dt.datetime.strptime(outlook_time, '%H%M').minute

bdt = dt.datetime(1999,5,3,my_hour,my_minute)
edt = dt.datetime(1999,5,4,my_hour,my_minute)

# Create list index of datetimes with a frequency of one per day
dts = pd.date_range(bdt, edt, freq='D')
bdts, edts = dts[:-1],dts[1:]

### Check outlook/PP data files

In [29]:
dates = [
    {
        'type': 'pp',
        'start': '11041965',
        'end': '12041965'
    },
    {
        'type': 'pp',
        'start': '03041974',
        'end': '04041974'
    },
    {
        'type': 'pp',
        'start': '31051985',
        'end': '01061985'
    },
    {
        'type': 'pp',
        'start': '03051999',
        'end': '04051999'
    },
    {
        'type': 'hum',
        'start': '27042011',
        'end': '28042011'
    }    
]

In [59]:
G = Gridder(tx=lons, ty=lats, dx=dx/100)

In [82]:
dists = []

for date in dates:
    # Start times
    sday = int(date['start'][:2].lstrip("0"))
    smon = int(date['start'][2:4].lstrip("0"))
    syear = int(date['start'][4:])
    
    # End times
    eday = int(date['end'][:2].lstrip("0"))
    emon = int(date['end'][2:4].lstrip("0"))
    eyear = int(date['end'][4:])
    
    # Create date array
    my_hour = dt.datetime.strptime(outlook_time, '%H%M').hour
    my_minute = dt.datetime.strptime(outlook_time, '%H%M').minute

    bdt = dt.datetime(syear,smon,sday,my_hour,my_minute)
    edt = dt.datetime(eyear,emon,eday,my_hour,my_minute)

    # Create list index of datetimes with a frequency of one per day
    dts = pd.date_range(bdt, edt, freq='D')
    bdts, edts = dts[:-1],dts[1:]
    
    edts = edts.map(lambda x: x.replace(hour=12,minute=0))
    
    if date['type'] == 'pp':
        regfile = f'../pp_forecasts/regProbs/{syear}{date["start"][2:4]}{date["start"][:2]}_1630.npz'
        reg_probs = np.load(regfile)['fcst']
        
        sigfile = f'../pp_forecasts/sigProbs/{syear}{date["start"][2:4]}{date["start"][:2]}_1630.npz'
        sig_probs = np.load(sigfile)['fcst']
    else:
        regfile = '../../sampleinputs/torn_day1_grib2_1200_20110427061047'
        with pg.open(regfile) as GRB:
            try:
                reg_probs = GRB[1].values.filled(-1)
            except AttributeError:
                reg_probs = GRB[1].values
        sigfile = '../../sampleinputs/sigtorn_day1_grib2_1200_20110427061047'
        with pg.open(sigfile) as GRB:
            try:
                sig_probs = GRB[1].values.filled(-1)
            except AttributeError:
                sig_probs = GRB[1].values
                
        reg_probs = reg_probs/100
        sig_probs = sig_probs/100
    
    ##
    ## Collect tornadoes in sig/mod
    ##
    
    double_sig_tors = []
    
    _df = df[(df['date_time'] >= bdts[0]) & (df['date_time'] < edts[0]) & (df['weight'] == 1.0)]
    if _df.empty:
        continue
    # print(f'From {bdt:%Y-%m-%d %H%M}z to {edt:%Y-%m-%d %H%M}z')
    
    lon1 = _df.slon.values
    lat1 = _df.slat.values
    lon2 = _df.elon.values
    lat2 = _df.elat.values
    
    # Find/remove/replace missing data
    keep = ~np.logical_or(lon1 == 0, lat1 == 0)
    lon1 = lon1[keep]
    lat1 = lat1[keep]
    lon2 = lon2[keep]
    lat2 = lat2[keep]
    lon2[lon2 == 0] = lon1[lon2 == 0]
    lat2[lat2 == 0] = lat1[lat2 == 0]
    
    # Mask sig grid where tor probs are below MDT threshold (less than 0.15)
    sig_probs = np.ma.masked_where(reg_probs < 0.15, sig_probs)
    
    # Loop through tornadoes
    for i in range(len(lat1)):
        # Grid tornadoes
        tornlines = G.grid_lines(sxs=lon1[i], sys=lat1[i], exs=lon2[i], eys=lat2[i])
        mags = _df['weight']
        fcst = G.make_empty_grid(dtype='float')
        for tornline, mag in zip(tornlines, mags):
            fcst[tornline] = mag
        
        if (np.max(np.ma.masked_where(sig_probs<0.05,fcst)) > 0):
        #if (np.max(np.ma.masked_where(vals<0.05,fcst)) > 0):
            double_sig_tors.append(i)
            
    rating_dist = _df.iloc[double_sig_tors,:].groupby('mag').count()['weight']/len(double_sig_tors)
    
    dists.append(rating_dist)

In [87]:
tot = dists[0]

for dist in dists[1:]:
   tot = tot.add(dist,fill_value=0)

print(f'Final Distribution: {tot/5}')

Final Distribution: mag
0    0.187347
1    0.250486
2    0.170708
3    0.167871
4    0.193852
5    0.029736
Name: weight, dtype: float64
